In [ ]:
import os
import pickle
import pandas as pd
import numpy as np

results = {}
datasets_conv = {
    'bpic2017_accepted': 'BPI2017',
    'bpic2012_accepted': 'BPI2012',
    'hospital_billing_2': 'Hospital Billing',
    'sepsis_cases_1': 'Sepsis Case 1',
    'sepsis_cases_2': 'Sepsis Case 2',
    'SimBank': 'SimBank'
}

dataset_dir = 'src/test_results'

for fname in os.listdir(dataset_dir):
    if fname.endswith('.pkl'):
        filepath = os.path.join(dataset_dir, fname)

        parts = fname.split("_")

        with open(filepath, 'rb') as f:
            data = pickle.load(f)

        dataset = datasets_conv[data['dataset']]
        quick_rej = data['quick_rejection_rates']   

        if dataset == "SimBank":
            if len(parts) > 1:
                model = parts[1]     
            else:
                model = "NoModel"
            key = f"{dataset}_{model}"
        else:
            key = dataset

        results[key] = quick_rej


In [ ]:
def format_value(val):
    return f"{val:.2f}" if isinstance(val, (int, float)) else "-"


# Step 2: Build function to extract alpha table
def build_alpha_table(alpha_idx, alpha_label):
    table = {}
    max_lag = max(len(r) for r in results.values())
    for dataset, lags in results.items():
        table[dataset] = [lags[i][alpha_idx] if i <= len(lags) else "-" for i in range(1,max_lag+1)]
    df = pd.DataFrame(table).T
    df.columns = [f"Lag {i+1}" for i in range(df.shape[1])]
    df.index.name = f"Dataset (α = {alpha_label})"
    df = df.applymap(format_value)
    return df

# Step 3: Generate tables for α = 1%, 5%, 10%
alpha_1_df = build_alpha_table(0, "1%")
alpha_5_df = build_alpha_table(1, "5%")
alpha_10_df = build_alpha_table(2, "10%")

# Step 4: Display tables
print("\n=== α = 1% ===")
print(alpha_1_df.to_string())
print("\n=== α = 5% ===")
print(alpha_5_df.to_string())
print("\n=== α = 10% ===")
print(alpha_10_df.to_string())



In [ ]:
print("\n% LaTeX Table for α = 5%")
print(alpha_5_df.to_latex(index=True, escape=False))